<a href="https://colab.research.google.com/github/AnupKumarGupta/Machine-Learning-Lab-Assignments/blob/master/Predict_Next_Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import numpy as np
import gensim
import string
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.callbacks import ModelCheckpoint

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Path to the dataset
path = "/content/sherlock_book.txt"

# Data Cleaning and Preprocessing 

# Restriction on the maximum length of sentence in terms of number of words
max_sentence_len = 40

# Open data file and read the data as lines
with open(path) as file_:
  docs = file_.readlines()

# Preprocessing the data
# 1. Converting the words into lower cases
# 2. Removing punctuation marks
# 3. Spliting the sentence into two if the max length exceeds
# 4. Remove the sentences which are empty or too short 
sentences = [[word for word in doc.lower().translate(str.maketrans('','',string.punctuation)).split()[:max_sentence_len]] for doc in docs]
sentences = [item for item in sentences if len(item) > 2]
print('Num of sentences:', len(sentences))

Num of sentences: 57351


In [3]:
# Training the Word2Vec model
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
pretrained_weights = word_model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape

# Embeddings shape will be : word count X size of vector
print('Result embedding shape:', pretrained_weights.shape)

2019-11-16 12:51:17,140 : INFO : collecting all words and their counts
2019-11-16 12:51:17,141 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-16 12:51:17,171 : INFO : PROGRESS: at sentence #10000, processed 114642 words, keeping 10118 word types
2019-11-16 12:51:17,200 : INFO : PROGRESS: at sentence #20000, processed 230211 words, keeping 13760 word types
2019-11-16 12:51:17,227 : INFO : PROGRESS: at sentence #30000, processed 344100 words, keeping 16512 word types
2019-11-16 12:51:17,254 : INFO : PROGRESS: at sentence #40000, processed 456890 words, keeping 18735 word types
2019-11-16 12:51:17,281 : INFO : PROGRESS: at sentence #50000, processed 570396 words, keeping 20963 word types
2019-11-16 12:51:17,304 : INFO : collected 22707 word types from a corpus of 653029 raw words and 57351 sentences
2019-11-16 12:51:17,305 : INFO : Loading a fresh vocabulary
2019-11-16 12:51:17,489 : INFO : effective_min_count=1 retains 22707 unique words (100% of origi

Result embedding shape: (22707, 100)


In [4]:
# Saving the model 
model_path = "sherlock_model"
word_model.save(model_path)

2019-11-16 12:52:57,846 : INFO : saving Word2Vec object under sherlock_model, separately None
2019-11-16 12:52:57,848 : INFO : not storing attribute vectors_norm
2019-11-16 12:52:57,852 : INFO : not storing attribute cum_table
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-11-16 12:52:58,212 : INFO : saved sherlock_model


In [0]:
# Retriving 'topn' words similar to 'word'
def get_most_similar(word,topn):
  try:
    print(word_model.most_similar(word,topn=topn))
  except:
    print("Word <",word,"> not in vocabulary")

In [0]:
# Method to return index of 'word' in the word2vec model
# Returns -1 if the 'word' is not present in the vocab 
def word2idx(word):
  try:
   index = word_model.wv.vocab[word].index
  except:
   index = -1
  return index   

# Method to return 'word' at a particular index in the word2vec model
# Returns empty string if the index is out of bounds
def idx2word(idx):
  try:
    word = word_model.wv.index2word[idx]
  except:
    word = ""  
  return word

In [0]:
# Preparing the input and coresspondings output for training the LSTMs
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])

In [8]:
# LSTM model 
lstm_model = tf.keras.models.Sequential()
lstm_model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
lstm_model.add(tf.keras.layers.LSTM(units=emdedding_size))
lstm_model.add(tf.keras.layers.Dense(units=vocab_size))
lstm_model.add(tf.keras.layers.Activation('softmax'))
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2019-11-16 12:52:58,744 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2019-11-16 12:52:58,774 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# Method to generate next samples using a temprature, 
# instead of vanila argmax
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
# Method to generate next 'count' words if a 'text' is provided
def generate_next(lstm_model, text, count=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(count):
    prediction = lstm_model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)  
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [0]:
# Creating a callback to save the LSTM model after every epoch
filepath="weights-improvement-{epoch:02d}-.hdf5"
checkpoint = ModelCheckpoint(filepath,verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
callbacks_list = [checkpoint]

In [0]:
# Training the LSTM model
lstm_model.fit(train_x, train_y,
          batch_size=128,
          epochs=25,
          callbacks =callbacks_list, verbose=1)